Twitter Sentiments Analysis

In [54]:
import selenium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import getpass
pd.options.display.max_colwidth = 1000


In [55]:
my_user="naqeeb2710"
my_pass=getpass.getpass()


In [56]:
search_item="Manish Sisodia"

In [57]:
path="C:\Program Files\driver\chromedriver.exe"
driver=webdriver.Chrome(path)
driver.get("https://twitter.com/i/flow/login")
sleep(3)

C:\Users\naqee\AppData\Local\Temp\ipykernel_24028\1402160771.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(path)


In [58]:
user_id=driver.find_element(By.XPATH,"//input[@type='text']")
user_id.send_keys(my_user)
user_id.send_keys(Keys.RETURN)

In [62]:
password=driver.find_element(By.XPATH,"//input[@type='password']")
password.send_keys(my_pass)
password.send_keys(Keys.RETURN)


Scrap Tweets

In [63]:
search_box=driver.find_element(By.XPATH,"//input[@data-testid='SearchBox_Search_Input']")
search_box.send_keys(search_item)
search_box.send_keys(Keys.RETURN)

In [65]:
all_tweets=set()
tweets=driver.find_elements(By.XPATH,"//div[@data-testid='tweetText']")

while True:
    for tweet in tweets:
        all_tweets.add(tweet.text)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(3)
    tweets=driver.find_elements(By.XPATH,"//div[@data-testid='tweetText']")
    if len(all_tweets)>100:
        break

    

In [66]:
all_tweets=list(all_tweets)
all_tweets[0]

'There are 3 types of lies — lies, damn lies and paid lies!Crores was spent on “meaningless advertisements “ in India by \n@ArvindKejriwal\n and now they have reached the International media too!! #ManishSisodia has lot of explaining to do on liquor policy \n@ManojTiwariMP\n \n@BJP4India'

Cleaning The Tweets

In [67]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naqee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naqee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
df=pd.DataFrame(all_tweets,columns=["Tweets"])
df.head()

,Tweets
0,"There are 3 types of lies — lies, damn lies and paid lies!Crores was spent on “meaningless advertisements “ in India by \n@ArvindKejriwal\n and now they have reached the International media too!! #ManishSisodia has lot of explaining to do on liquor policy \n@ManojTiwariMP\n \n@BJP4India"
1,"Delhi | Today we are speaking for AAP party, but when the opposition is raided, AAP chooses silence: RJD Manoj Jha on raid at Delhi Dy CM Manish Sisodia's house"
2,"Today Manish Sisodia was declared the best Education Minister in the world but a CBI team reached his residence to conduct a raid. So, there'll be a lot of obstacles. It wasn't easy to appear on the front page of New York Times &bring an Education Revolution in Delhi: CM Kejriwal"
3,#ManishSisodia forget’s his memory his memory at CBI offie
4,How can we forget where it all started?\n\nNew York Times & Khaleej Times following foot steps of \n@ppbajpai\n & \n@ArvindKejriwal\n exploiting them!\n\nBahut Krantikari #ManishSisodia


In [69]:
stp_words=stopwords.words('english')
print(stp_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [70]:
def clean_tweet(tweet): 
    cleanTweet=re.sub(r"@[a-zA-Z0-9]+","",tweet)
    cleanTweet=re.sub(r"#[a-zA-Z0-9\s]+","",cleanTweet)
    cleanTweet=" ".join(word for word in cleanTweet.split() if word not in stp_words)
    return cleanTweet

In [71]:
df['CleanTweets']=df['Tweets'].apply(clean_tweet)
df.head()

,Tweets,CleanTweets
0,"There are 3 types of lies — lies, damn lies and paid lies!Crores was spent on “meaningless advertisements “ in India by \n@ArvindKejriwal\n and now they have reached the International media too!! #ManishSisodia has lot of explaining to do on liquor policy \n@ManojTiwariMP\n \n@BJP4India","There 3 types lies — lies, damn lies paid lies!Crores spent “meaningless advertisements “ India reached International media too!!"
1,"Delhi | Today we are speaking for AAP party, but when the opposition is raided, AAP chooses silence: RJD Manoj Jha on raid at Delhi Dy CM Manish Sisodia's house","Delhi | Today speaking AAP party, opposition raided, AAP chooses silence: RJD Manoj Jha raid Delhi Dy CM Manish Sisodia's house"
2,"Today Manish Sisodia was declared the best Education Minister in the world but a CBI team reached his residence to conduct a raid. So, there'll be a lot of obstacles. It wasn't easy to appear on the front page of New York Times &bring an Education Revolution in Delhi: CM Kejriwal","Today Manish Sisodia declared best Education Minister world CBI team reached residence conduct raid. So, there'll lot obstacles. It easy appear front page New York Times &bring Education Revolution Delhi: CM Kejriwal"
3,#ManishSisodia forget’s his memory his memory at CBI offie,’s memory memory CBI offie
4,How can we forget where it all started?\n\nNew York Times & Khaleej Times following foot steps of \n@ppbajpai\n & \n@ArvindKejriwal\n exploiting them!\n\nBahut Krantikari #ManishSisodia,How forget started? New York Times & Khaleej Times following foot steps & exploiting them! Bahut Krantikari


Analysing And Visulaing The Data